In [15]:
import pandas as pd
import numpy as np

In [11]:
# Load the data and replace '.' with NaN
SOFR_index_df = pd.read_csv('SOFR.csv')
SOFR_index_df['SOFR'] = SOFR_index_df['SOFR'].replace('.', np.nan)

# Convert SOFR column to float
SOFR_index_df['SOFR'] = SOFR_index_df['SOFR'].astype(float)

# Convert the 'DATE' column to datetime
SOFR_index_df['DATE'] = pd.to_datetime(SOFR_index_df['DATE'])

# Generate a full date range
full_date_range = pd.date_range(start=SOFR_index_df['DATE'].min(), end=SOFR_index_df['DATE'].max())

# Reindex the DataFrame to include all dates
SOFR_index_df = SOFR_index_df.set_index('DATE').reindex(full_date_range).reset_index()
SOFR_index_df.columns = ['DATE', 'SOFR']

# Create the 'business_day' column: 0 if SOFR is NaN, 1 if SOFR is not NaN
SOFR_index_df['business_day'] = np.where(SOFR_index_df['SOFR'].isnull(), 0, 1)

# Replace all NaN values in 'SOFR' with the last available business rate
SOFR_index_df['SOFR'].ffill(inplace=True)


# Set the initial value for index_approx on 2019-08-29
initial_date = pd.to_datetime('2019-08-30')
initial_value = 1.0316827

# # Set the initial value for the first row matching the initial_date
# SOFR_index_df.loc[SOFR_index_df['DATE'] == initial_date, 'index_approx'] = initial_value


SOFR_index_df['calendar_days_applicable'] = 0  # Default value

# Loop through and set calendar_days_applicable based on business_day
for i in range(len(SOFR_index_df) - 1):
    if SOFR_index_df.loc[i, 'business_day'] == 1:
        count = 1
        for j in range(i + 1, len(SOFR_index_df)):
            if SOFR_index_df.loc[j, 'business_day'] == 0:
                count += 1
            else:
                break
        SOFR_index_df.loc[i, 'calendar_days_applicable'] = count

# # Calculate the subsequent values of index_approx
# for i in range(1, len(SOFR_index_df)):
#     if pd.notnull(SOFR_index_df.loc[i, 'multiple_2']):
#         SOFR_index_df.loc[i, 'index_approx_2'] = SOFR_index_df.loc[i-1, 'index_approx_2'] * SOFR_index_df.loc[i, 'multiple_2']

# Calculate 'multiple_2' using the previous row's SOFR and calendar_days_applicable values
SOFR_index_df['multiple_2'] = 1 + (SOFR_index_df['SOFR'].shift(1) / 100) * (SOFR_index_df['calendar_days_applicable'].shift(1) / 360)

# Display the updated DataFrame
SOFR_index_df.head(10)



# Initialize the index_approx column with NaN
SOFR_index_df['index_approx_2'] = np.nan



# Set the initial value for the first row matching the initial_date
SOFR_index_df.loc[SOFR_index_df['DATE'] == initial_date, 'index_approx_2'] = initial_value

for i in range(1, len(SOFR_index_df)):
    if SOFR_index_df.loc[i - 1, 'business_day'] == 1:  # Only update on business days
        # Ensure the previous value exists before multiplication
        if pd.notnull(SOFR_index_df.loc[i-1, 'index_approx_2']):
            SOFR_index_df.loc[i, 'index_approx_2'] = SOFR_index_df.loc[i-1, 'index_approx_2'] * SOFR_index_df.loc[i, 'multiple_2']
    else:
        SOFR_index_df.loc[i, 'index_approx_2'] = SOFR_index_df.loc[i - 1, 'index_approx_2']
  

# Display the updated DataFrame
SOFR_index_df.head(20)
SOFR_index_df.to_csv('SOFR_index.csv', index=False)

SOFR_index_df_clean = SOFR_index_df[SOFR_index_df['business_day'] >= 1]

SOFR_index_df_clean.to_csv('SOFR_index.csv', index=False)


In [12]:
SOFR_index_df_clean

,DATE,SOFR,business_day,calendar_days_applicable,multiple_2,index_approx_2
0,2019-08-29,2.12,1,1,NaN,NaN
1,2019-08-30,2.16,1,4,1.000059,1.031683
5,2019-09-03,2.17,1,1,1.000000,1.031930
6,2019-09-04,2.21,1,1,1.000060,1.031993
7,2019-09-05,2.21,1,1,1.000061,1.032056
...,...,...,...,...,...,...
1821,2024-08-23,5.33,1,3,1.000147,1.154522
1824,2024-08-26,5.34,1,1,1.000000,1.155035
1825,2024-08-27,5.35,1,1,1.000148,1.155206
1826,2024-08-28,5.35,1,1,1.000149,1.155378
